In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import vibrationtesting as vt
from vibrationtesting import sos_modal
import matplotlib.pyplot as plt
import scipy
import numpy as np
import scipy.io as sio
import array_to_latex as a2t
np.set_printoptions(precision = 9, linewidth = 220, suppress = True)

# Eigen Vectors from Testing 

In [2]:
mat_contents=sio.loadmat('1stNaturalFreq.mat')
U1 = mat_contents['U1']
#U1 = vt.real_modes(U1, autorotate = True)
mat_contents=sio.loadmat('2ndNaturalFreq.mat')
U2 = mat_contents['U2']
#U2 = vt.real_modes(U2, autorotate = True)
mat_contents=sio.loadmat('3rdNaturalFreq.mat')
U3 = mat_contents['U3']
#U3 = vt.real_modes(U3, autorotate = True)
mat_contents=sio.loadmat('4rthNaturalFreq.mat')
U4 = mat_contents['U4']
#U4 = vt.real_modes(U4, autorotate = True)
mat_contents=sio.loadmat('5thNaturalFreq.mat')
U5 = mat_contents['U5']
#U5 = vt.real_modes(U5, autorotate = True)
mat_contents=sio.loadmat('6thNaturalFreq.mat')
U6 = mat_contents['U6']
#U6 = vt.real_modes(U6, autorotate = True)

Psi_1 = np.array(U1)
Psi_2 = np.array(U2)
Psi_3 = np.array(U3)
Psi_4 = np.array(U4)
Psi_5 = np.array(U5)
Psi_6 = np.array(U6)
Psi_1=np.column_stack((Psi_1,Psi_2,Psi_3,Psi_4,Psi_5,Psi_6))


Psi_abs = np.abs(Psi_1)*np.real(np.sign(Psi_1))
Psi_1 = Psi_abs
Psi_1



array([[   5.241707706,  -27.127615081,   52.843453543,  107.210143005,   71.139095015, -269.817342785],
       [   4.687209363,  -26.945738191,   52.183186313,   -7.83341953 ,   71.411534453,  -40.625538373],
       [   4.368421773,  -25.121917234,   51.597488464,  -99.159212806,   69.26683148 ,  238.962801192],
       [   3.519108382,   -7.307635452,  -36.871073212,  112.432870575,  -75.970925544, -209.599864678],
       [   3.99369394 ,   -8.550399951,  -37.014491536,   34.494279202,  -65.780768515,  -57.021770026],
       [   4.110844861,   -7.792721262,  -38.903453864,  -97.14023879 ,  -68.483622372,  185.356134448],
       [   3.912418166,   17.983583519,  -33.877855743,  101.225337752,   30.716200755,   95.968190423],
       [   3.359713741,   17.158904587,  -35.675183371,   37.881749912,   46.813407339,  -11.419977031],
       [   3.303682334,   17.92248139 ,  -34.000342956,  -88.24444254 ,   41.527236694, -103.954353298],
       [   2.277194422,   17.704562416,   27.531465665,

# *Note*: 
* SEREP technique was applied using WFEM, the results were exact as the full model only when no: of Modeshapes required to keep were in sequential order but we don't need the 3rd modeshape that FEA model is giving. So, SEREP may not be useful for this case.


# Extension of modes from Modal Analysis to All DOF of a Finite Element Model 


In [3]:
mat_contents=sio.loadmat('WingBeamforMAC.mat')
K = mat_contents['K']
M = mat_contents['M']
measured = np.array([[2,8,14,20,26,32,38,44,50,56,62,68,74,80,86,92,98,104]])
omega=13.54
vt.mode_expansion_from_model(Psi_1, omega, M, K, measured)

NotImplementedError: Reshaping not implemented for csr_matrix.

In [26]:
K.shape

(141, 141)

In [27]:
M.shape

(141, 141)

In [6]:
import scipy.sparse as spsp

In [7]:
spsp.issparse(M)

True

It appears that the issue below in the vibrationtesting module is that sparse matrices are not handled. Slicing, multiplying, and inverting will all have to be done separately for sparse matrices. See the [sparse matrices manual](https://docs.scipy.org/doc/scipy/reference/sparse.html) . 

In the short term, converting them to full matrices will be good enough. In the long term, there will need to be parallel paths, one for sparse matrices, one for non-sparse. The appropriate format using scipy sparse matrices appears to be  [`lil_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.lil_matrix.html#scipy.sparse.lil_matrix). On fix might be to rewrite, and automatically convert all matrices to this format before the math, then convert to the originating format afterwards to make the use happy. 

#  Tried using Extension of modes from Modal Analysis to All DOF of a Finite Element Model after applying Guyan reduction

In [28]:
mat_contents=sio.loadmat('WingBeamforMAC.mat')
K = mat_contents['Kr']
M = mat_contents['Mr']
measured = np.array([[1,6,11,16,21,26,31,36,41,46,51,56,61,66,71]])
omega=13.54
vt.mode_expansion_from_model(Psi_1, omega, M, K, measured)

NotImplementedError: Reshaping not implemented for csr_matrix.

In [29]:
K.shape

(75, 75)

In [30]:
M.shape

(75, 75)

# full FE eigen vectors

In [34]:
Psi_1=fms
Psi_1.shape

(141, 75)

# full Testing eigen vectors

In [37]:
Psi_full.shape

NameError: name 'Psi_full' is not defined

# MAC after DOF expansion

In [36]:
nummodes = Psi_full.shape[1]
MAC = np.zeros((nummodes, nummodes))
if Psi_full.shape == Psi_2.shape:
        for i in np.arange(nummodes):
            for j in np.arange(nummodes):
                MAC[i, j] = (abs(np.conj(Psi_full[:, i]) @ Psi_2[:, j])**2 /
                             (np.conj(Psi_full[:, i]) @ Psi_full[:, i] *
                              np.conj(Psi_2[:, j]) @ Psi_2[:, j]))
else:
        print('Mode shape arrays must have the same size.')
    

NameError: name 'Psi_full' is not defined

# FE model Correction

In [ ]:

model_correction_direct(Psi_full, omega, M, K, method='Baruch')